In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
from sklearn.model_selection import KFold

In [3]:
iris = pd.read_csv('/home/fernando/Documents/ai/iris.data', header=None)

In [35]:
def clasificacion(base_datos):
    name = base_datos.iloc[:, -1]

    #FOR PARA ESTABLECER LOS INDICES DE CADA CLASE
    i = 0
    index_list = []
    indexes = []
    for j in range(base_datos.shape[0]):
        if name[j] != name[i]:
            indexes.append(i)
            indexes.append(j)
            index_list.append(indexes)
            i = j
            indexes = []
    indexes.append(i)
    indexes.append(j+1)
    index_list.append(indexes)    
        
    #FOR PARA GUARDAR LOS DATOS DE UNA CLASE COMO ELEMENTOS (TIPO PANDAS) DE UNA LISTA
    dataset = []
    for e in index_list:
        dataset.append(base_datos.iloc[e[0]:e[1], :])
    
    return dataset

In [ ]:
def media(clase):
    sum = np.sum(clase, axis=0)
    dimensiones = clase.shape[0]
    media_lista = np.array([])
        
    for i in sum:
        media_lista = np.append(media_lista, i/dimensiones)
        
    return media_lista

In [ ]:
def varianza(clase):
    
    dimensiones = clase.shape[1]
    longitud = clase.shape[0]
    med = media(clase)
    

    varianza_lista = []
    
    for i in range(dimensiones):
        varianza_suma = 0
        for j in range(longitud):
            varianza_suma += (clase[j][i] - med[i])**2
        varianza_lista.append(varianza_suma/(longitud-1))
            
    return varianza_lista

In [280]:
def probabilidad(punto, dataset):
        
    productos = []
    i = 0
    t = 0
    total_elements = 0
    for clase in dataset:
        total_elements += clase.shape[0]

    for i in dataset:
        med = media(i)
        vari = varianza(i)
        prob = 1
        for j in range(int(dataset[0].shape[1])):
            prob *= (math.e**((-(punto[j] - med[j])**2)/(2*vari[j])))/math.sqrt(2*math.pi*vari[j])*(dataset[t].shape[0]/total_elements) 
        productos.append(prob)
        t += 1
    
    probabilidades = []
    for i in range(len(productos)):
        probabilidades.append(productos[i]/sum(productos))
    
    prob = probabilidades
    
    name = ''
    if (prob[0] > prob[1] and prob[0] > prob[2]):
        name = 'Iris-setosa'
    elif (prob[1] > prob[0] and prob[1] > prob[2]):
        name = 'Iris-versicolor'
    elif (prob[2] > prob[0] and prob[2] > prob[1]):
        name = 'Iris-virginica'

#    print(name)
    
    return name

In [105]:
def kfolds(dataset, k = 10):
    kf = KFold(n_splits=k)

    #FOR QUE GENERA LA ESTRUCTURA DE MI LISTA
    index_kfolds = []
    for i in range(k):
        index_kfolds.append([])

    #FOR QUE VA LLENANDO MI LISTA CON CADA FOLD
    i = 0
    for clase in dataset:
        for training_index, test_index in kf.split(clase):
            item = []
            item.append(test_index)
            item.append(training_index)
            index_kfolds[i].append(item)
            i +=1
        i = 0
        
    return index_kfolds

In [90]:
def convert_numpy(test_index, dataset):
    dimensions = dataset.shape[1] -1 
    list_dimensions = [i for i in range(dimensions)]
    margin = dataset.iloc[0].name
    
    for e in test_index:
        dataset = dataset.drop(e+margin)
    
    return dataset.iloc[:, list_dimensions].values

In [176]:
def convert_point(index, clase):
    dimensions = clase.shape[1] -1
    margin = clase.iloc[0].name

    item = []
    for i in range(dimensions):
        item.append(clase[i][index+margin])
    
    return item

In [281]:
#Cross Validation
dataset = clasificacion(iris)
folds = kfolds(dataset)
n = 1
for fold in folds:
    i = 0
    
    # FOR PARA CONVERTIR NUESTROS INDICES EN NUMPY ARRAY
    numpy_dataset = []
    for clase in fold:
        numpy_dataset.append(convert_numpy(clase[0], dataset[i]))
        i += 1
    
    print('#Test', n)
    n += 1
    
    #FOR PARA SACAR LAS PROBABILIDADES
    i = 0
    accuracy_test = []
    accuracy_train = []

    for clase in fold:
        right_test = 0
        total_test = 0
        right_train = 0
        total_train = 0
        
        #FOR PARA EVALUAR EL TEST_SET
        for point_index in clase[0]:
            if probabilidad(convert_point(point_index, dataset[i]), numpy_dataset)==dataset[i].iloc[point_index,-1]:
                right_test +=1
                total_test += 1
            else:
                total_test += 1
        accuracy_test.append(right_test/total_test)
                
        #FOR PARA EVALUAR EL TRAIN_SET
        for point_index in clase[1]:
            if probabilidad(convert_point(point_index, dataset[i]), numpy_dataset)==dataset[i].iloc[point_index,-1]:
                right_train += 1
                total_train += 1
            else:
                total_train += 1
        accuracy_train.append(right_train/total_train)
        
        i += 1
        
    #CALCULO MEDIAS
    media_test = sum(accuracy_test)/len(accuracy_test)
    media_train = sum(accuracy_train)/len(accuracy_train)
    
    #CALCULO DESVIACION
    desviacion_test = 0
    for e in accuracy_test:
        desviacion_test += (e-media_test)**2
    desviacion_test = math.sqrt(desviacion_test/len(accuracy_test))

    desviacion_train = 0
    for e in accuracy_train:
        desviacion_train += (e-media_train)**2
    desviacion_train = math.sqrt(desviacion_train/len(accuracy_train))

    
    print('\t\tTest \t\t Train\nmedia:\t\t{:5.4f}\t\t{:5.4f}\ndesviacion:\t{:5.5f}\t\t{:5.4f}'.format(media_test, media_train, desviacion_test, desviacion_train))
    print()


#Test 1
		Test 		 Train
media:		0.9333		0.9630
desviacion:	0.09428		0.0277

#Test 2
		Test 		 Train
media:		0.9333		0.9630
desviacion:	0.09428		0.0277

#Test 3
		Test 		 Train
media:		1.0000		0.9556
desviacion:	0.00000		0.0363

#Test 4
		Test 		 Train
media:		0.9333		0.9556
desviacion:	0.09428		0.0314

#Test 5
		Test 		 Train
media:		0.9333		0.9630
desviacion:	0.09428		0.0277

#Test 6
		Test 		 Train
media:		0.9333		0.9630
desviacion:	0.09428		0.0277

#Test 7
		Test 		 Train
media:		0.8667		0.9704
desviacion:	0.18856		0.0210

#Test 8
		Test 		 Train
media:		1.0000		0.9481
desviacion:	0.00000		0.0378

#Test 9
		Test 		 Train
media:		1.0000		0.9630
desviacion:	0.00000		0.0277

#Test 10
		Test 		 Train
media:		1.0000		0.9630
desviacion:	0.00000		0.0277

